In [2]:
from casadi import SX, inv, vertcat, Function, integrator, sin, cos, diag, skew

In [3]:
#Equations of motion expressed in BODY

#6 DOF states vectors
u = SX.sym('u') 
v = SX.sym('v') 
w = SX.sym('w')
v_nb = vertcat(u,v,w) #body-fixed linear velocity

p = SX.sym('p') 
q = SX.sym('q') 
r = SX.sym('r')
w_nb = vertcat(p,q,r) #body-fixed angular velocity

x_nb = vertcat(v_nb,w_nb)

m = SX.sym('m')  # Mass

I_x = SX.sym('I_x') # moment of inertia x entry
I_y = SX.sym('I_y') # moment of inertia y entry
I_z = SX.sym('I_z') # moment of inertia z entry
I_zx = SX.sym('I_zx') # product of inertia zx entry 
I_yx = SX.sym('I_yx') # product of inertia yx entry 
I_zy = SX.sym('I_zy') # product of inertia zy entry 

x_g = SX.sym('x_g')  # Center of gravity, x-axis
y_g = SX.sym('y_g')  # Center of gravity, z-axis
z_g = SX.sym('z_g')  # Center of gravity, z-axis

x_b = SX.sym('x_b')  # Center of buoyancy, x-axis
y_b = SX.sym('y_b')  # Center of buoyancy, z-axis
z_b = SX.sym('z_b')  # Center of buoyancy, z-axis


# Added Mass Forces in a Rotating Coordinate System

#Hydrodynamic System Inertia Matrix
M_A = -SX.sym('M_A',6,6)

# Hydrodynamic Coriolis and centripetal matrix
C_A = SX.zeros(6,6)
a = M_A@x_nb
a_n =  SX.sym('a_n',3) 
a_n[0] = a[0]
a_n[1] = a[1]
a_n[2] = a[2]

b_n =  SX.sym('b_n',3)
b_n[0] = a[3]
b_n[1] = a[4]
b_n[2] = a[5]

C_A[3:,:3] = -skew(a_n)
C_A[:3,3:] = -skew(a_n)
C_A[3:,3:] = -skew(b_n)

C_A

SX(@1=0, @2=((((((M_A_2*u)+(M_A_8*v))+(M_A_14*w))+(M_A_20*p))+(M_A_26*q))+(M_A_32*r)), @3=((((((M_A_1*u)+(M_A_7*v))+(M_A_13*w))+(M_A_19*p))+(M_A_25*q))+(M_A_31*r)), @4=((((((M_A_0*u)+(M_A_6*v))+(M_A_12*w))+(M_A_18*p))+(M_A_24*q))+(M_A_30*r)), @5=((((((M_A_5*u)+(M_A_11*v))+(M_A_17*w))+(M_A_23*p))+(M_A_29*q))+(M_A_35*r)), @6=((((((M_A_4*u)+(M_A_10*v))+(M_A_16*w))+(M_A_22*p))+(M_A_28*q))+(M_A_34*r)), @7=((((((M_A_3*u)+(M_A_9*v))+(M_A_15*w))+(M_A_21*p))+(M_A_27*q))+(M_A_33*r)), 
[[@1, @1, @1, @1, (-@2), @3], 
 [@1, @1, @1, @2, @1, (-@4)], 
 [@1, @1, @1, (-@3), @4, @1], 
 [@1, (-@2), @3, @1, (-@5), @6], 
 [@2, @1, (-@4), @5, @1, (-@7)], 
 [(-@3), @4, @1, (-@6), @7, @1]])

In [4]:
# Rigid body kinetics

# body inertia tensor
I_rb = SX.sym('I_rb',3,3)
I_rb[0,0] = I_x
I_rb[1,1] = I_y
I_rb[2,2] = I_z
I_rb[1,0] = -I_yx
I_rb[2,0] = -I_zx
I_rb[2,1] = -I_zy
I_rb[0,1] = I_rb[1,0]
I_rb[0,2] = I_rb[2,0]
I_rb[1,2] = I_rb[2,1]

# Rigid-Body System Inertia Matrix
M_rb = SX(6, 6)  # Create a 6x6 symbolic matrix

# Manually populate the matrix
M_rb[:3, :3] = m*SX.eye(3)
M_rb[3:, 3:] = I_rb

S_v  =  SX.sym('S_v',3)  # inertia skew symmetric component
S_v[0] = m*x_g
S_v[1] = m*y_g
S_v[2] = m*z_g

S = skew(S_v)

M_rb[3:, :3] = S
M_rb[:3, 3:] = -S

print(M_rb)


# Coriolis-Centripetal Matrix from System Inertia Matrix
# Lagrangian Parameterization
C_rb = SX(6, 6)  # Create a 6x6 symbolic matrix
# Manually populate the matrix
c_m = SX(3, 3) 

c_m[0,0] = -m*(y_g*q + z_g*r)
c_m[1,0] = m*(x_g*q - w)
c_m[2,0] = m*(x_g*r + v)

c_m[0,1] = m*(y_g*p + w)
c_m[1,1] = -m*(z_g*r + x_g*p)
c_m[2,1] = m*(y_g*r - u)

c_m[0,2] = m*(z_g*p - v)
c_m[1,2] = m*(z_g*q + u)
c_m[2,2] = -m*(x_g*p + y_g*q)

c_I_v = SX.sym('c_I_v',3)
c_I_v[0] = I_zx*r + I_yx*q - I_x*p
c_I_v[1] = I_zy*r + I_yx*p - I_y*q
c_I_v[2] = I_zy*q + I_zx*p - I_z*r
c_I = skew(c_I_v)

C_rb[:3, :3] = SX.zeros(3,3)
C_rb[:3, 3:] = -c_m.T
C_rb[3:, :3] = c_m
C_rb[3:, 3:] = c_I

C_rb
# Inertia Matrix ROV & AUV

@1=0, @2=(m*z_g), @3=(m*y_g), @4=(m*x_g), @5=(-I_yx), @6=(-I_zx), @7=(-I_zy), 
[[m, 00, 00, @1, @2, (-@3)], 
 [00, m, 00, (-@2), @1, @4], 
 [00, 00, m, @3, (-@4), @1], 
 [@1, (-@2), @3, I_x, @5, @6], 
 [@2, @1, (-@4), @5, I_y, @7], 
 [(-@3), @4, @1, @6, @7, I_z]]


SX(@1=0, @2=(m*((y_g*q)+(z_g*r))), @3=(m*((x_g*q)-w)), @4=(m*((x_g*r)+v)), @5=(m*((y_g*p)+w)), @6=(m*((z_g*r)+(x_g*p))), @7=(m*((y_g*r)-u)), @8=(m*((z_g*p)-v)), @9=(m*((z_g*q)+u)), @10=(m*((x_g*p)+(y_g*q))), @11=(((I_zy*q)+(I_zx*p))-(I_z*r)), @12=(((I_zy*r)+(I_yx*p))-(I_y*q)), @13=(((I_zx*r)+(I_yx*q))-(I_x*p)), 
[[@1, @1, @1, @2, (-@3), (-@4)], 
 [@1, @1, @1, (-@5), @6, (-@7)], 
 [@1, @1, @1, (-@8), (-@9), @10], 
 [(-@2), @5, @8, @1, (-@11), @12], 
 [@3, (-@6), @9, @11, @1, (-@13)], 
 [@4, @7, (-@10), (-@12), @13, @1]])

In [7]:
#assuming Starboard–port symmetrical underwater vehicles yg=0 and Ixy = Iyz = 0
M = M_rb + M_A
M

SX(@1=(m*z_g), @2=(m*y_g), @3=(m*x_g), 
[[(m-M_A_0), (-M_A_6), (-M_A_12), (-M_A_18), (@1-M_A_24), (-(@2+M_A_30))], 
 [(-M_A_1), (m-M_A_7), (-M_A_13), (-(@1+M_A_19)), (-M_A_25), (@3-M_A_31)], 
 [(-M_A_2), (-M_A_8), (m-M_A_14), (@2-M_A_20), (-(@3+M_A_26)), (-M_A_32)], 
 [(-M_A_3), (-(@1+M_A_9)), (@2-M_A_15), (I_x-M_A_21), (-(I_yx+M_A_27)), (-(I_zx+M_A_33))], 
 [(@1-M_A_4), (-M_A_10), (-(@3+M_A_16)), (-(I_yx+M_A_22)), (I_y-M_A_28), (-(I_zy+M_A_34))], 
 [(-(@2+M_A_5)), (@3-M_A_11), (-M_A_17), (-(I_zx+M_A_23)), (-(I_zy+M_A_29)), (I_z-M_A_35)]])

In [16]:
# Hydrostatics of Submerged Vehicles
# restoring forces

W = SX.sym('W')  # weight
B = SX.sym('B')  # buoyancy

thet = SX.sym('thet')  
phi = SX.sym('phi')  

g = SX(6, 1)

g[0,0] = (W - B)*sin(thet)
g[1,0] = -(W - B)*cos(thet)*sin(phi)
g[2,0] = -(W - B)*cos(thet)*cos(phi)
g[3,0] = -(y_g*W -y_b*B)*cos(thet)*cos(phi) + (z_g*W -z_b*B)*cos(thet)*sin(phi)
g[4,0] = (z_g*W -z_b*B)*sin(thet) + (x_g*W -x_b*B)*cos(thet)*cos(phi)
g[5,0] = -(x_g*W -x_b*B)*cos(thet)*sin(phi) - (y_g*W -y_b*B)*sin(thet)

#For neutrally buoyant vehicles W = B


In [ ]:
# Damping will be nonlinear and coupled for an underwater vehicle moving in 6 DOFs at high speed

# <xUabsU>	Quadratic drag in surge

# <yVabsV>	Quadratic drag in sway

# <zWabsW>	Quadratic drag in heave

# <kPabsP>	Quadratic drag in roll

# <mQabsQ>	Quadratic drag in pitch

# <nRabsR>	Quadratic drag in yaw



# vehicle is performing a noncoupled motion, we can assume a diagonal structure
X_u = SX.sym('X_u') # <xU>	Linear drag in surge
Y_v = SX.sym('Y_v') # <yV>	Linear drag in sway
Z_w = SX.sym('Z_w') # <zW>	Linear drag in heave
K_p = SX.sym('K_p') # <kP>	Linear drag in roll
M_q = SX.sym('M_q') # <mQ>	Linear drag in pitch
N_r = SX.sym('N_r') # <nR>	Linear drag in yaw

